## <font color='blue'>Sistemas de Recomendação para Filmes</font>

### Instalando e Carregando Pacotes

In [1]:
# Imports
import ast
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.options.mode.chained_assignment = None

## Carregando e Compreendendo os Dados

In [2]:
# Dataset de filmes
filmes = pd.read_csv("dados/dataset_filmes.csv")

In [3]:
# Shape
filmes.shape

(4803, 20)

In [4]:
# Amostra dos dados
filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
# Dataset de elenco dos filmes
elenco = pd.read_csv("dados/dataset_elenco.csv")

In [6]:
# Shape
elenco.shape

(4803, 4)

In [7]:
# Amostra dos dados
elenco.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Organização dos Dados de Texto

> Existe uma relação entre os 2 datasets. Vamos fazer o merge pela coluna comum entre eles, a coluna title.

In [8]:
# Merge dos dataframes
filmes = filmes.merge(elenco, on = 'title')

In [9]:
# Shape
filmes.shape

(4809, 23)

In [10]:
# Amostra dos dados
filmes.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [11]:
# Info
filmes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [12]:
# Filtrando o dataframe e mantendo somente as colunas consideradas mais relevantes
filmes = filmes[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [13]:
filmes.head(10)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...","[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...","[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,38757,Tangled,When the kingdom's most wanted-and most charmi...,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...","[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
7,99861,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...","[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...","[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
9,209112,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 7002...","[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...","[{""credit_id"": ""553bf23692514135c8002886"", ""de..."


## Análise Exploratória e Limpeza dos Dados

In [14]:
# Checando valores ausentes
filmes.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [15]:
# Removemos as 3 linhas com valores ausentes
filmes.dropna(inplace = True)

In [16]:
# Checando valores ausentes
filmes.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [17]:
# Checando se há valores duplicados
filmes.duplicated().sum()

0

In [18]:
# Shape
filmes.shape

(4806, 7)

## Processamento de Texto com Abstract Syntax Trees

In [19]:
# Função de conversão
# Este loop itera sobre os elementos de uma estrutura de dados. 
# Usamos ast.literal_eval(obj) para avaliar/converter a string obj em uma estrutura de dados Python. 
def converter(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [20]:
# Aplicando a função ao dataset, na coluna de gênero do filme
filmes['genres'] = filmes['genres'].apply(converter)

In [21]:
# Aplicando a função ao dataset, na coluna de keywords do filme
filmes['keywords'] = filmes['keywords'].apply(converter)

Aplicar o ast à coluna de cast (elenco) mas extrair sobre 3 membros do elenco e não sobre o elenco todo.

In [22]:
# Função de conversão
def converter3(obj):
    
    # Cria uma lista vazia chamada L para armazenar os nomes.
    L = []  
    
    # Inicializa um contador para controlar o número de elementos adicionados à lista.
    counter = 0  
    
    # Usa ast.literal_eval para avaliar a string 'obj' e iterar sobre os elementos resultantes.
    for i in ast.literal_eval(obj):  
        
        # Verifica se o contador é diferente de 3.
        if counter != 3:  
            
            # Adiciona o valor da chave 'name' do dicionário atual à lista L.
            L.append(i['name'])  
            
            # Incrementa o contador.
            counter += 1  
        else:
            
            # Interrompe o loop se o contador chegar a 3.
            break  
    
    # Retorna a lista L com até 3 nomes.
    return L  

In [23]:
# Aplicamos então a função ao nosso dataset, na coluna de cast (elenco) do filme
filmes['cast'] = filmes['cast'].apply(converter3)

A função abaixo vai extrair o nome do diretor do filme.

In [24]:
# Função de conversão
def fetch_director(obj):
    
    # Cria uma lista vazia chamada L para armazenar o nome do diretor.
    L = []
    
    # Converte a string 'obj' em um objeto Python e itera sobre seus elementos.
    for i in ast.literal_eval(obj):
        
        # Verifica se o trabalho ('job') do elemento atual é 'Director'.
        if i['job'] == 'Director':
            
            # Adiciona o nome do diretor (valor da chave 'name') à lista L.
            L.append(i['name'])
            
            # Interrompe o loop após encontrar o primeiro diretor.
            break
            
    # Retorna a lista L, contendo o nome do diretor.
    return L

In [25]:
# Aplicamos então a função ao nosso dataset, na coluna de crew (equipe) do filme
filmes['crew'] = filmes['crew'].apply(fetch_director)

In [26]:
filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


## Limpeza dos Dados de Texto

In [27]:
# Separamos a string da coluna overview por espaço em branco
filmes['overview'] = filmes['overview'].apply(lambda x:x.split())

In [28]:
# Replace de espaço por vazio (remove o espaço)
filmes['genres'] = filmes['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [29]:
# Replace de espaço por vazio (remove o espaço)
filmes['keywords'] = filmes['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [30]:
# Replace de espaço por vazio (remove o espaço)
filmes['cast'] = filmes['cast'].apply(lambda x:[i.replace(" ","") for i in x])

In [31]:
# Replace de espaço por vazio (remove o espaço)
filmes['crew'] = filmes['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [32]:
filmes.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


### Preparando os Dados Para Vetorização em Outro Espaço Vetorial

In [33]:
# Criamos a coluna de tags, nesse caso um vetor de strings com os valores das colunas
filmes['tags'] = filmes['overview'] + \
                        filmes['genres'] + \
                        filmes['keywords'] + \
                        filmes['cast'] + \
                        filmes['crew']

In [34]:
# Dataset final
filmes_final = filmes[['movie_id', 'title', 'tags']]

In [35]:
filmes_final.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [36]:
# Join das strings para simplificar o vetor
filmes_final['tags'] = filmes_final['tags'].apply(lambda x:" ".join(x))

In [37]:
# Coloca tudo em minúsculo para evitar diferença de palavras maiúsculo/minúsculo
filmes_final['tags'] = filmes_final['tags'].apply(lambda x:x.lower())

In [38]:
filmes_final.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


## Parse e Vetorização

In [39]:
# Criamos o parser
parser_ps = PorterStemmer()

In [40]:
# Função de stemming
def stem(text):
    
    # Cria uma lista vazia chamada y para armazenar as palavras após o stemming.
    y = []
    
    # Divide a string de entrada 'text' em palavras e itera sobre elas.
    for i in text.split():
        
        # Realiza o stemming na palavra atual 'i' e adiciona o resultado à lista y.
        y.append(parser_ps.stem(i))
    
    # Retorna as palavras processadas como uma string, unindo-as com espaços.
    return " ".join(y)

In [41]:
# Aplica a função à coluna de tags
filmes_final['tags'] = filmes_final['tags'].apply(stem)

In [42]:
filmes_final.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [43]:
# Cria o vetorizador com no máximo 5000 atributos
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [44]:
# Cria os vetores para as tags
vectors = cv.fit_transform(filmes_final['tags']).toarray()

In [45]:
len(cv.get_feature_names_out())

5000

In [46]:
type(vectors)

numpy.ndarray

## Cálculo de Distância dos Vetores

A célula abaixo trabalha com os vetores no espaço de dimensão para calcular a distância matemática entre eles usando a distância de cosseno (cosine_similarity).

In [47]:
# Calcula a similaridade entre os vetores calculando as distâncias entre eles
similaridades = cosine_similarity(vectors)

## Construindo o Sistema de Recomendação

In [48]:
# Função para o sistema de recomendação
def sistema_recomendacao(movie):
    
    # Obtém o índice do filme passado como argumento (o que o usuário assistiu)
    index = filmes_final[filmes_final['title'] == movie].index[0]
    
    # Verificamos então os filmes com vetores de menor distância para o filme passado como argumento
    distances = sorted(list(enumerate(similaridades[index])), reverse = True, key = lambda x: x[1])
    
    # E então consideramos os 5 filmes com menor distância, ou seja, maior similaridade
    for i in distances[1:6]:
        print(filmes_final.iloc[i[0]].title)

## Aplicando o Sistema de Recomendação

In [49]:
# Quais as recomendações de filmes para quem assistiu ao filme: Avengers: Age of Ultron?
sistema_recomendacao('The Dark Knight Rises')

The Dark Knight
Batman Returns
Batman
Batman Forever
Batman Begins


In [50]:
# Quais as recomendações de filmes para quem assistiu ao filme: Jurassic World?
sistema_recomendacao("Pirates of the Caribbean: At World's End")

Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
Life of Pi
20,000 Leagues Under the Sea


In [51]:
# Quais as recomendações de filmes para quem assistiu ao filme: The Hobbit: The Battle of the Five Armies?
sistema_recomendacao('The Hobbit: The Battle of the Five Armies')

The Hobbit: An Unexpected Journey
The Hobbit: The Desolation of Smaug
The Lord of the Rings: The Two Towers
The Lord of the Rings: The Return of the King
The Lord of the Rings: The Fellowship of the Ring


In [52]:
# Quais as recomendações de filmes para quem assistiu ao filme: John Carter?
sistema_recomendacao('John Carter')

Riddick
Krrish
The Other Side of Heaven
The Legend of Hercules
Get Carter
